# MLP
Test of simple MLP models with different amount of neurons, hidden layers and size of input vector
## Load Data

In [99]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
 
import pandas as pd
import numpy as np

from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam


import matplotlib.pyplot as plt

In [100]:
df = pd.read_csv("data.csv")
df_diff = pd.DataFrame(df.iloc[1:].values - df.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

In [101]:
class SequentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, n_dots=1, n_dot_parameters=4):
        self.n_dot_parameters = n_dot_parameters
        self.n_dots = n_dots
        all_data = self.__make_stack(dataframe)
        self.X_ = all_data[:, :-n_dot_parameters]
        self.y_ = all_data[:, -n_dot_parameters:]
        
    def __len__(self):
        return len(self.X_)
    
    def __getitem__(self, idx):
        return self.X_[idx], self.y_[idx]
    
    def __make_stack(self, df: pd.DataFrame):
        stacks = [df.iloc[:-self.n_dots]] + [df.iloc[i:].values if (self.n_dots == i) else df.iloc[i:-(self.n_dots - i)].values for i in range(1, self.n_dots + 1)]
        return torch.tensor(np.hstack(stacks), dtype=torch.float32)
    
    def plotData(self, i = 0, f = -1):
        fig = px.line_3d(x = self.X_[i:f, 1], y = self.X_[i:f, 2], z = self.X_[i:f, 3])
        fig.show()

In [102]:
N_DOTS = 10
N_DOT_PARAMETERS = 4

In [103]:
dataset = SequentDataset(df_diff[0:2500], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
test = SequentDataset(df_diff[2500:3000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
dataset.plotData()

In [104]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

## Create Model

In [105]:
class MLPModel(nn.Module):
    def __init__(self, input_dots_amount, input_dot_parameters_amount, output_dots_amount, output_dot_parameters_amount, hidden_layers_lengths, dropout_probability=0, activation=nn.ReLU()):
        super(MLPModel, self).__init__()
        self.layers = nn.ModuleList()
        layer_lengths = [input_dots_amount * input_dot_parameters_amount] + hidden_layers_lengths + [output_dots_amount * output_dot_parameters_amount]
        #self.dropout = nn.Dropout(dropout_probability)
        self.activation = activation
        for i in range(1, len(layer_lengths)):
            self.layers.append(nn.Linear(layer_lengths[i - 1], layer_lengths[i]))
    
    
    def forward(self, X: torch.Tensor):
        X = X.clone()
        for layer in self.layers[:-1]:
            X = layer(X)
            X = self.activation(X)
            #X = self.dropout(X)
        return self.layers[-1](X)

In [106]:
model = MLPModel(input_dots_amount=N_DOTS, input_dot_parameters_amount=N_DOT_PARAMETERS, output_dots_amount=1, output_dot_parameters_amount=4, hidden_layers_lengths=[100, 20, 50])
loss_model = nn.MSELoss(reduction='mean')
opt = Adam(model.parameters(), lr=0.001)

In [107]:
model

MLPModel(
  (layers): ModuleList(
    (0): Linear(in_features=40, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=4, bias=True)
  )
  (activation): ReLU()
)

In [108]:
EPOCH = 100
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    train_loop = tqdm(train_loader,leave=False)
    train_loss = []
    for X, y in train_loader:
        pred = model(X)
        loss = loss_model(pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)
    
    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

  0%|          | 0/63 [00:00<?, ?it/s]

Epoch [1/100], train_loss = 0.0955, val_loss = 0.0031


Epoch [2/100], train_loss = 0.0016, val_loss = 0.0014


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch [3/100], train_loss = 0.0014, val_loss = 0.0014


...

Epoch [97/100], train_loss = 0.0009, val_loss = 0.0010


Epoch [98/100], train_loss = 0.0009, val_loss = 0.0010


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch [99/100], train_loss = 0.0009, val_loss = 0.0010


Epoch [100/100], train_loss = 0.0009, val_loss = 0.0010


In [110]:
dataset[0]

(tensor([ 1.0000,  0.0058, -0.0736, -0.0145,  1.0000, -0.0485,  0.0673, -0.0123,
          1.0000,  0.0051, -0.0111,  0.0394,  1.0000,  0.0530, -0.0326,  0.0036,
          1.0000, -0.0099, -0.0226,  0.0326,  1.0000, -0.0433, -0.0087, -0.0676,
          1.0000,  0.0295,  0.0495,  0.0312,  1.0000,  0.0010, -0.0797, -0.0361,
          1.0000, -0.0609,  0.0613,  0.0677,  1.0000,  0.0761, -0.0849, -0.0710]),
 tensor([ 1.0000, -0.0818,  0.0145,  0.0086]))

In [115]:
start, _  = dataset[0]
predictions = []
model.eval()

for i in range(1,500):
    with torch.no_grad():
        pred = model(start)
        predictions.append(pred)
        start = torch.hstack([start[N_DOT_PARAMETERS:], pred]) 
        # start = pred   

In [116]:
predictions

[tensor([ 1.0018, -0.0303,  0.0341,  0.0224]),
 tensor([ 0.9987, -0.0174, -0.0072,  0.0145]),
 tensor([ 0.9991,  0.0086, -0.0053, -0.0161]),
 tensor([ 9.9732e-01, -9.4911e-03, -5.8499e-03,  3.9279e-05]),
 tensor([ 0.9987, -0.0092,  0.0045, -0.0023]),
 tensor([ 9.9782e-01, -8.3084e-04, -5.8525e-03,  2.3517e-03]),
 tensor([ 9.9758e-01, -1.1769e-02, -1.8197e-03,  4.2422e-04]),
 tensor([ 0.9985, -0.0057, -0.0100, -0.0048]),
 tensor([ 9.9717e-01, -1.2217e-02,  6.1056e-03, -8.3101e-04]),
 tensor([ 0.9979, -0.0025, -0.0050, -0.0083]),
 tensor([ 0.9975, -0.0214,  0.0056,  0.0091]),
 tensor([ 0.9972, -0.0071, -0.0045, -0.0069]),
 tensor([ 0.9971, -0.0110,  0.0012, -0.0034]),
 tensor([ 9.9686e-01, -1.4811e-02,  1.9003e-04,  5.6057e-04]),
 tensor([ 0.9970, -0.0134,  0.0013, -0.0025]),
 tensor([ 9.9666e-01, -1.4752e-02, -2.8927e-04, -2.5805e-03]),
 tensor([ 0.9968, -0.0151,  0.0030, -0.0022]),
 tensor([ 0.9966, -0.0162,  0.0013, -0.0024]),
 tensor([ 0.9965, -0.0171,  0.0032, -0.0014]),
 tensor([ 0

In [117]:
p_x = pd.DataFrame(map(lambda t: t.detach().tolist(), predictions), columns=["Time", "X", "Y", "Z"])
_, y  = dataset[:500]
p_y = pd.DataFrame(y.detach().numpy(), columns=["Time", "X", "Y", "Z"])

p_x = p_x.cumsum()
p_y = p_y.cumsum()

In [118]:
p_y

,Time,X,Y,Z
0,1.0,-0.081768,0.014475,0.008595
1,2.0,-0.077035,0.001584,0.014794
2,3.0,-0.020011,0.025885,0.003004
3,4.0,-0.073985,-0.022478,0.012483
4,5.0,-0.010897,-0.019937,0.033617
...,...,...,...,...
495,496.0,1.675227,-1.896814,-1.111645
496,497.0,1.703794,-1.865375,-1.094644
497,498.0,1.747322,-1.843668,-1.114741
498,499.0,1.728526,-1.834327,-1.117131


In [119]:
p_x["C"] = "Predict"
p_y["C"] = "True"

d = pd.concat([p_x,p_y],axis=0)

In [120]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C')
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()